In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda, GRU, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD, RMSprop
import math
import pickle
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [5]:
index = "dowjones"
index = "frankfurt"
with open(f"../data/{index}_calculated/periods750_250_240.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
def normalize_data(df):
    """normalize a dataframe."""
    mean = df.mean(axis=1)
    std = df.std(axis=1)
    df = df.sub(mean, axis=0)
    df = df.div(std, axis=0)
    df = df.values
    return df
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [7]:
i = 7
timestep = 240

In [8]:
# x_train = dataset[0][i][0]['AMZN'].values * 1000
# y_train = dataset[0][i][1]['AMZN'].values * 1.0
# x_test = dataset[1][i][0]['AMZN'].values * 1000
# y_test = dataset[1][i][1]['AMZN'].values * 1.0

# x_train = dataset[0][i][0].values
# x_train = (x_train - x_train.mean())/x_train.std()
# y_train = dataset[0][i][1].values * 1.0
# x_test = dataset[1][i][0].values
# x_test = (x_test - x_test.mean())/x_test.std()
# y_test = dataset[1][i][1].values * 1.0

# x_train = dataset[0][i][0].values * 1000
# x_test = dataset[1][i][0].values * 1000

x_train = dataset[0][i][0].values
x_test = dataset[1][i][0].values

scaler = StandardScaler().fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# x_train = normalize_data(dataset[0][i][0])
# x_test = normalize_data(dataset[1][i][0])

# y_train = get_one_hot(dataset[0][i][1].values, 2) * 1.0
# y_test = get_one_hot(dataset[1][i][1].values, 2) * 1.0
y_train = to_categorical(dataset[0][i][1].values, 2)
y_test = to_categorical(dataset[1][i][1].values, 2)

In [9]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {x_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (750, 62)
y train shape: (750, 62, 2)
x test shape: (490, 62)
y test shape: (490, 62, 2)


In [10]:
# The second range will be looped first
# x_series = [x_train[i:i+240] for i in range(750 - 240)]
# y_series = [y_train[i+240] for i in range(750 - 240)]
x_series = [x_train[i:i+timestep, j] for i in range(x_train.shape[0] - timestep) for j in range(x_train.shape[1])]
y_series = [y_train[i+timestep, j] for i in range(y_train.shape[0] - timestep) for j in range(y_train.shape[1])]
x = np.array(x_series)
y = np.array(y_series)
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (31620, 240)
y shape: (31620, 2)


In [11]:
x = np.reshape(x, (x.shape[0], x.shape[1], 1))
print(f"x shape: {x.shape}")


x shape: (31620, 240, 1)


In [12]:
dropout_rate = 0.1
# expected input data shape: (batch_size, timesteps, data_dim)
regressor = Sequential()

# regressor.add(Bidirectional(LSTM(units=25, input_shape=(timestep, 1), dropout=dropout_rate)))
regressor.add(LSTM(units=25, input_shape=(timestep, 1), return_sequences = True,dropout=dropout_rate))
regressor.add(LSTM(units=100, return_sequences = True,dropout=dropout_rate))
regressor.add(LSTM(units=100, return_sequences = True,dropout=dropout_rate))
regressor.add(LSTM(units=100, input_shape=(timestep, 1), dropout=dropout_rate))
# regressor.add(LSTM(units=25, batch_input_shape=(527, timestep, 1), dropout=dropout_rate, stateful=False))
# regressor.add(LSTM(units=25, batch_input_shape=(527, timestep, 1), dropout=dropout_rate))
# regressor.add(LSTM(units=25, return_sequences = True,dropout=dropout_rate, stateful=False))
# regressor.add(LSTM(units=25, return_sequences = True,dropout=dropout_rate, stateful=False))
# regressor.add(LSTM(units=25, dropout=dropout_rate, stateful=False))
# regressor.add(LSTM(units=25, input_shape=(timestep, 1), dropout=dropout_rate))
# regressor.add(GRU(units=25, input_shape=(timestep, 1), dropout=dropout_rate))
# regressor.add(Dense(100, input_shape=(timestep, ), activation='relu'))
# regressor.add(Dense(100, activation='relu'))
regressor.add(Dense(2, activation='softmax'))
regressor.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 240, 25)           2700      
_________________________________________________________________
lstm_2 (LSTM)                (None, 240, 100)          50400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 240, 100)          80400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 214,102
Trainable params: 214,102
Non-trainable params: 0
_________________________________________________________________


In [13]:
# result = regressor.fit(x, y, epochs=1000,batch_size=527, validation_split=0.2, shuffle=False, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=200),
#              ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])
result = regressor.fit(x, y, epochs=1000,batch_size=527, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=200),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])
# regressor.fit(x, y, epochs=1000,batch_size=500, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=20),
#              ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])


Train on 25296 samples, validate on 6324 samples
Epoch 1/1000
25296/25296 [==============================] - 52s 2ms/step - loss: 0.6930 - acc: 0.5104 - val_loss: 0.6935 - val_acc: 0.5016
Epoch 2/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6928 - acc: 0.5145 - val_loss: 0.6933 - val_acc: 0.5016
Epoch 3/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6929 - acc: 0.5144 - val_loss: 0.6934 - val_acc: 0.5016
Epoch 4/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6927 - acc: 0.5142 - val_loss: 0.6932 - val_acc: 0.5022
Epoch 5/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6928 - acc: 0.5147 - val_loss: 0.6932 - val_acc: 0.5019
Epoch 6/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6928 - acc: 0.5146 - val_loss: 0.6937 - val_acc: 0.5016
Epoch 7/1000
25296/25296 [==============================] - 51s 2ms/step - loss: 0.6928 - acc: 0.5140 - val_loss: 0.6

KeyboardInterrupt: 

In [ ]:
plt.plot(result.history["val_acc"])
plt.plot(result.history["acc"])

In [ ]:
plt.plot(result.history["val_loss"])
plt.plot(result.history["loss"])